<a href="https://colab.research.google.com/github/rexian/ML/blob/main/langchain/nvidia/langchain_prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install langchain-core langgraph>0.2.27
!pip install langchain_community langchain_huggingface
!pip install langchain-nvidia-ai-endpoints

In [2]:
import getpass
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA

if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA AI Endpoints: ")
nvidia_model = ChatNVIDIA(model="meta/llama3-70b-instruct")

Enter API key for NVIDIA AI Endpoints: ··········


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
def run_llm_chain(model_id: str,
                  prompt:PromptTemplate,
                  input: str,
                  temperature: float = 1e-3,
                  max_length: int = 4096,
                  do_sample: bool = True) -> str:
    """
    Initializes a HuggingFaceHub with specified parameters and runs an LLMChain with a given question.

    Args:
    repo_id (str): The repository ID for the HuggingFace model.
    temperature (float, optional): The temperature for text generation. Default is 0.5.
    max_length (int, optional): The maximum length of the generated text. Default is 64.

    Returns:
    str: The result from running the LLMChain.
    """
    # Define the prompt template within the function
    prompt = prompt
    llm = ChatNVIDIA(model=model_id)
    llm_chain = prompt | llm | StrOutputParser()
    return llm_chain.invoke({"input":input})

- Avoid ambiguity

- Use distinct delimiters (triple quotes, backticks, angle brackets) to separate instructions clearly.

In [5]:
text_to_summarize = f"""It's important to give clear and specific instructions \
when asking a model to perform a task. This will help guide the model towards \
the desired output, and minimize the chances of receiving irrelevant or \
incorrect responses. It's important to note that writing a clear prompt  \
doesn't necessarily mean writing a short one. In fact, longer prompts can  \
often provide more clarity and context for the model, resulting in more \
detailed and relevant outputs.
"""

template = """Summarize the text delimited by triple backticks into a single sentence: ```{input}```.
Summary:
"""

prompt = PromptTemplate.from_examples(
    examples=[template],
    input_variables=["input"],
    prefix = "<s>[INST]",
    suffix="[/INST]"
    )

prompt.pretty_print()

<s>[INST]

Summarize the text delimited by triple backticks into a single sentence: ```{input}```.
Summary:


[/INST]


In [8]:
instruct_model_id = "meta/llama3-70b-instruct"
result = run_llm_chain(model_id = instruct_model_id,
              prompt = prompt,
              input = text_to_summarize)
print(result)

When asking a model to perform a task, it is crucial to provide clear and specific instructions, which can include longer prompts that offer more context and clarity, to guide the model towards the desired output and reduce the likelihood of irrelevant responses.


**Request Structured Output**

- Prioritize clarity: Ask for outputs in structured formats (like JSON, XML).

- Structured data is simpler to interpret and use.

- Defining the desired format upfront leads to more organized
results.

In [9]:
text = """Create some imaginary movie titles, complete with directors and genres.
Format the information in JSON, using the keys: 'movie_id', 'title', 'director', and 'genre'.
"""
prompt = PromptTemplate.from_examples(
    examples=[text],
    input_variables=[],
    prefix = "<s>[INST]",
    suffix="[/INST]"
    )
prompt.pretty_print()

<s>[INST]

Create some imaginary movie titles, complete with directors and genres.
Format the information in JSON, using the keys: 'movie_id', 'title', 'director', and 'genre'.


[/INST]


In [12]:
result = run_llm_chain(model_id = instruct_model_id,
              prompt = prompt,
              input = text_to_summarize)
print(result)

Here are 5 imaginary movie titles with directors and genres in JSON format:

```
[
  {
    "movie_id": 1,
    "title": "Echoes in the Dark",
    "director": "Lucas Brooks",
    "genre": "Thriller"
  },
  {
    "movie_id": 2,
    "title": "Stardust Serenade",
    "director": "Ava Moreno",
    "genre": "Romantic Sci-Fi"
  },
  {
    "movie_id": 3,
    "title": "Burning Inferno",
    "director": "Jaxon Lee",
    "genre": "Action"
  },
  {
    "movie_id": 4,
    "title": "The Lost City of Eldarado",
    "director": "Evelyn Stone",
    "genre": "Adventure"
  },
  {
    "movie_id": 5,
    "title": "Ghosts of Memories",
    "director": "Rylan Patel",
    "genre": "Horror"
  }
]
```

Let me know if you'd like me to generate more!


**Process for Verifying Conditions**

* **Spot the Criteria**: Look for specific requirements, like a set of instructions.
* **Assess Compliance**: Examine the text to see if it meets these criteria.
* **Customized Reply**: React accordingly - proceed if criteria are met, or take a different route if not.
* **Transparent Feedback**: Clearly explain if conditions were met and the logic behind your response.



In [20]:
text = """To reset your password, first, navigate to the settings page. Click \
on the 'Security' tab, and then select 'Change Password.' Enter your current \
password, followed by your new password twice for confirmation. Click on the \
'Save Changes' button to update your password.
"""

input_template = """You will be provided with text enclosed in angle brackets (<>). Determine \
if the text contains a procedural guide.  If it does, convert the guide into a \
numbered list format with a title:

<Title>

1. ...
2. ...
3. ...
4. ...

If the text does not contain a procedural guide, respond with 'No procedural guide found.'

<{input}>
"""

In [21]:
prompt = PromptTemplate.from_examples(
    examples=[input_template],
    input_variables=["input"],
    prefix = "<s>[INST]",
    suffix="[/INST]"
    )
prompt.pretty_print()

<s>[INST]

You will be provided with text enclosed in angle brackets (<>). Determine if the text contains a procedural guide.  If it does, convert the guide into a numbered list format with a title:

<Title>

1. ...
2. ...
3. ...
4. ...

If the text does not contain a procedural guide, respond with 'No procedural guide found.'

<{input}>


[/INST]


In [22]:
result = run_llm_chain(model_id = instruct_model_id,
              prompt = prompt,
              input = text)
print(result)

Here is the converted procedural guide:

**Resetting Your Password**

1. Navigate to the settings page.
2. Click on the 'Security' tab.
3. Select 'Change Password.'
4. Enter your current password, followed by your new password twice for confirmation.
5. Click on the 'Save Changes' button to update your password.


**Strategies for Complex Queries**

* Detail Steps & Format
* Break Down Tasks
* Logical Sequencing
* Think Before Concluding

In [23]:
template = """Your task is to determine if a student's solution is correct or incorrect.\

To solve the problem do the following:

- First, work out your own solution to the problem including the final total.

- Then compare your solution to the student's solution and evaluate if the student's\
solution is correct or not.

Don't decide if the student's solution is correct until  you have done the problem yourself.

Question: ```{input}```

"""

suffix = """Respond with the following format:

Question: ```question here```

Student's solution: ```student's solution here```

Actual solution: ```steps to work out the solution and your solution here```

Is the student's solution the same as actual solution just calculated:```yes or no```

Student grade: ```correct or incorrect```

Think step-by-step, and assess provide your solution to the question.

My solution is: [/INST]
"""


question = """```I'm building a solar power installation and I need help working out the financials:
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year,\
and an additional $10 / square foot

What is the total cost for the first year of operations as a function of the number of square feet.```

Student's solution:```Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x

Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000```
"""

prompt = PromptTemplate.from_examples(
    examples=[template],
    input_variables=["input"],
    prefix = "<s>[INST]",
    suffix=suffix
    )
prompt.pretty_print()

<s>[INST]

Your task is to determine if a student's solution is correct or incorrect.
To solve the problem do the following:

- First, work out your own solution to the problem including the final total.

- Then compare your solution to the student's solution and evaluate if the student'ssolution is correct or not.

Don't decide if the student's solution is correct until  you have done the problem yourself.

Question: ```{input}```



Respond with the following format:

Question: ```question here```

Student's solution: ```student's solution here```

Actual solution: ```steps to work out the solution and your solution here```

Is the student's solution the same as actual solution just calculated:```yes or no```

Student grade: ```correct or incorrect```

Think step-by-step, and assess provide your solution to the question.

My solution is: [/INST]



In [25]:
result = run_llm_chain(model_id = instruct_model_id,
              prompt = prompt,
              input = question)
print(result)

Here is my response:

Question: ``I'm building a solar power installation and I need help working out the financials:
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year,and an additional $10 / square foot

What is the total cost for the first year of operations as a function of the number of square feet.``

Student's solution: ``Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x

Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000``

Actual solution:

Let's break down the costs:

1. Land cost: $100/square foot, so the total land cost is 100x
2. Solar panel cost: $250/square foot, so the total solar panel cost is 250x
3. Maintenance cost: $100,000 per year (fixed cost) + $10/square foot, so the total maintenance cost is 100,000 + 10x

Total cost = Land cost + Solar panel cost + Mai

# Summary

1. **Clarity is Essential**: Use clear delimiters to avoid ambiguity and ensure precise responses from the model.

2. **Request Structured Output**: Ask for data in organized formats like JSON or XML for ease of interpretation and use.

3. **Process for Verifying Conditions**:
   - **Spot the Criteria**: Identify specific requirements in the text.
   - **Assess Compliance**: Check if the text meets these criteria.
   - **Customized Reply**: Respond based on the fulfillment of conditions.
   - **Transparent Feedback**: Clearly communicate the outcome and reasoning.

4. **Strategies for Complex Queries**:
   - **Detail Steps & Format**: Define steps and output format for complex tasks.
   - **Break Down Tasks**: Use step-by-step instructions for better processing.
   - **Logical Sequencing**: Ask for a logical flow before the final answer.
   - **Think Before Concluding**: Encourage thorough processing before conclusions.

5. **Balance Context and Clarity**: Provide essential information without overloading, allowing the model to request more context if needed.